# SRCNN

In [1]:
import torch

#"mps" if torch.backends.mps.is_available() else "cpu"

In [1]:
from train import main

In [2]:
main()

Load all datasets successfully
Build SRCNN model successfully.
Define all loss functions successfully.
Define all optimizer functions successfully.
Check whether the pretrained model is restored...


/Users/mariusgrybe/Workspace/upscalify/srcnn/train.py:82: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()
/usr/local/Caskroom/miniconda/base/envs/research-2/lib/python3.12/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Users/mariusgrybe/Workspace/upscalify/srcnn/train.py:251: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/Caskroom/miniconda/base/envs/research-2/lib/python3.12/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Epoch: [1][   0/1380]	Time  0.076 ( 0.076)	Data  0.000 ( 0.000)	Loss 0.271058 (0.271058)
Epoch: [1][ 200/1380]	Time  0.134 ( 0.060)	Data  0.105 ( 0.024)	Loss 0.144848 (0.216770)
Epoch: [1][ 400/1380]	Time  0.135 ( 0.060)	Data  0.103 ( 0.026)	Loss 0.028113 (0.135008)
Epoch: [1][ 600/1380]	Time  0.154 ( 0.062)	Data  0.122 ( 0.028)	Loss 0.022078 (0.097752)
Epoch: [1][ 800/1380]	Time  0.170 ( 0.064)	Data  0.130 ( 0.029)	Loss 0.023316 (0.078972)
Epoch: [1][1000/1380]	Time  0.169 ( 0.066)	Data  0.134 ( 0.029)	Loss 0.026632 (0.067526)
Epoch: [1][1200/1380]	Time  0.159 ( 0.066)	Data  0.126 ( 0.030)	Loss 0.019718 (0.059906)


/Users/mariusgrybe/Workspace/upscalify/srcnn/train.py:327: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Test: [0/5]	Time  0.323 ( 0.323)	PSNR 22.17 (22.17)	SSIM 0.7363 (0.7363)
Test: [1/5]	Time  0.160 ( 0.242)	PSNR 17.90 (20.04)	SSIM 0.5910 (0.6637)
Test: [2/5]	Time  0.197 ( 0.227)	PSNR 23.28 (21.12)	SSIM 0.6612 (0.6628)
Test: [3/5]	Time  0.186 ( 0.217)	PSNR 20.79 (21.04)	SSIM 0.7213 (0.6774)
Test: [4/5]	Time  1.024 ( 0.378)	PSNR 22.54 (21.34)	SSIM 0.7658 (0.6951)
 * Time 0.38 PSNR 21.34 SSIM 0.70


Epoch: [2][   0/1380]	Time  0.040 ( 0.040)	Data  0.000 ( 0.000)	Loss 0.024221 (0.024221)
Epoch: [2][ 200/1380]	Time  0.249 ( 0.073)	Data  0.154 ( 0.031)	Loss 0.023143 (0.020837)
Epoch: [2][ 400/1380]	Time  0.246 ( 0.081)	Data  0.174 ( 0.024)	Loss 0.020422 (0.020482)
Epoch: [2][ 600/1380]	Time  0.192 ( 0.084)	Data  0.150 ( 0.021)	Loss 0.017995 (0.020144)
Epoch: [2][ 800/1380]	Time  0.165 ( 0.084)	Data  0.131 ( 0.019)	Loss 0.018815 (0.019825)
Epoch: [2][1000/1380]	Time  0.169 ( 0.083)	Data  0.130 ( 0.021)	Loss 0.017543 (0.019588)
Epoch: [2][1200/1380]	Time  0.157 ( 0.081)	Data  0.125 ( 0.023)	L

KeyboardInterrupt: 